Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the Dataset.

Refere to a complete list of [built-in transforms](https://pytorch.org/docs/stable/torchvision/transforms.html)


### On Images

CenterCrop, Grayscale, Pad, RandomAffine RandomCrop, RandomHorizontalFlip, RandomRotationResize, Scale

### On Tensors

LinearTransformation, Normalize, RandomErasing

### Conversion

**ToPILImage**: from tensor or ndrarray

**ToTensor**: from numpy.ndarray or PILImage

### Generic

Use Lambda 

### Custom

Write own class

### Compose multiple Transforms

```
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)]
```

In [5]:
import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np

In [23]:
class WineDataset(Dataset):
    """
    Implement a custom Dataset: inherit Dataset
    Implement __init__ , __getitem__ , and __len__
    """
    def __init__(self, transform=None):
        # Initialize data, download, etc.
        # read with numpy or pandas
        xy = np.loadtxt('https://raw.githubusercontent.com/python-engineer/pytorchTutorial/master/data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        # Here the first column is the class label, the rest are the features
        # no need to convert to tensors from numpy
        self.x_data = xy[:, 1:] # size [n_samples, n_features]
        self.y_data = xy[:, [0]] # size [n_samples, 1]
        
        self.transform = transform
        
    def __getitem__(self, index):
        # support indexing such that dataset[i] can be used to get i-th sample
        # implement this python function for indexing
        
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):
        # we can call len(dataset) to return the size, so this can be used
        # as an iterator
        return self.n_sample

Let's define our a couple of custom transformers

In [29]:
class ToTensor:
    # convert ndarrays to tensors
    def __call__(self, sample):
        # Unpack sample
        inputs, targets = sample
        
        # convert to tensor
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
class MultTransform:
    # takes factor as multiplicant
    def __init__(self, factor):
        self.factor = factor
    
    # make it callable
    def __call__(self, sample):
        # Unpack sample
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets
    
def print_tensor_info(features, labels):
    print('features: {}'.format(features))
    print('labels  :{}'.format(labels))

In [33]:
# Let's now access our without transform dataset
dataset = WineDataset()
first_dataset = dataset[0]
features, labels = first_dataset
print_tensor_info(features, labels)

features: [1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
labels  :[1.]


In [31]:
# Let's now access our with transform dataset
dataset = WineDataset(transform=ToTensor())
first_dataset = dataset[0]
features, labels = first_dataset
print_tensor_info(features, labels)

features: tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
labels  :tensor([1.])


Use compose functionality to create mulitple transformers for the data

In [35]:
for factor in range(4):
    composed = torchvision.transforms.Compose([ToTensor(), MultTransform(factor+1)])
    dataset = WineDataset(transform=composed)
    first_dataset = dataset[0]
    features, labels = first_dataset
    print_tensor_info(features, labels)

features: tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
labels  :tensor([1.])
features: tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])
labels  :tensor([1.])
features: tensor([4.2690e+01, 5.1300e+00, 7.2900e+00, 4.6800e+01, 3.8100e+02, 8.4000e+00,
        9.1800e+00, 8.4000e-01, 6.8700e+00, 1.6920e+01, 3.1200e+00, 1.1760e+01,
        3.1950e+03])
labels  :tensor([1.])
features: tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03])
labels  :tensor([1.])
